In [1]:
69965-34978

34987

In [3]:
86019- 84254

1765

In [4]:
34987/1765

19.822662889518412

In [25]:
import requests
def fetch_brain_viewer_details(biosample_id):
    bfidic = {
        "222": 100,
        "244": 116,
    }
    bfi_value = bfidic.get(str(biosample_id))
    if not bfi_value:
        return None
    url = f"http://dev2adi.humanbrain.in:8000/GW/getBrainViewerDetails/IIT/V1/SS-{bfi_value}:-1:-1"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error if the response code isn't 200
        return response.json()  # Or response.text based on the expected data format
    except requests.exceptions.RequestException as e:
        return None

In [ ]:
# Import all necessary libraries for image processing
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageOps, ImageFilter, ImageEnhance
import os
import sys
import json
import requests
from pathlib import Path
import glob
import shutil
from typing import Optional, Tuple, Union, List
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib to display plots inline
%matplotlib inline

# Configure matplotlib for better plots
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

print("✅ All necessary libraries imported successfully!")
print("📦 Available libraries:")
print("- OpenCV (cv2) for advanced image processing")
print("- NumPy for numerical operations")
print("- Pandas for data manipulation")
print("- Matplotlib for plotting and visualization")
print("- PIL (Pillow) for image handling")
print("- OS and Path utilities for file operations")
print("- Requests for HTTP operations")
print("- JSON for data serialization")
print("- Glob for file pattern matching")
print("- Shutil for file operations")
print("- Typing for type hints")
print("\n🚀 Ready for image processing tasks!")

In [ ]:
# Additional utility functions for image processing
def check_image_info(image_path: str) -> dict:
    """
    Get comprehensive information about an image file.
    
    Args:
        image_path: Path to the image file
        
    Returns:
        Dictionary containing image information
    """
    try:
        # Check if file exists
        if not os.path.exists(image_path):
            return {"error": f"File not found: {image_path}"}
        
        # Get file information
        file_size = os.path.getsize(image_path)
        file_ext = Path(image_path).suffix.lower()
        
        # Load with PIL to get basic info
        with Image.open(image_path) as img:
            info = {
                "file_path": image_path,
                "file_size_bytes": file_size,
                "file_size_mb": round(file_size / (1024 * 1024), 2),
                "file_extension": file_ext,
                "image_mode": img.mode,
                "image_size": img.size,
                "width": img.width,
                "height": img.height,
                "format": img.format,
                "has_transparency": img.mode in ('RGBA', 'LA') or 'transparency' in img.info,
            }
            
            # Calculate aspect ratio
            info["aspect_ratio"] = round(img.width / img.height, 3)
            
            # Get color information if RGB
            if img.mode == 'RGB':
                img_array = np.array(img)
                info["mean_color"] = np.mean(img_array, axis=(0, 1)).astype(int).tolist()
                info["std_color"] = np.std(img_array, axis=(0, 1)).astype(int).tolist()
        
        return info
        
    except Exception as e:
        return {"error": f"Error processing image: {str(e)}"}

def create_image_grid(images: List[Union[str, Image.Image]], 
                     titles: Optional[List[str]] = None,
                     grid_size: Optional[Tuple[int, int]] = None,
                     figsize: Tuple[int, int] = (15, 10)) -> None:
    """
    Display multiple images in a grid layout.
    
    Args:
        images: List of image paths or PIL Image objects
        titles: List of titles for each image
        grid_size: Tuple of (rows, cols) for grid layout
        figsize: Figure size for the plot
    """
    n_images = len(images)
    
    if grid_size is None:
        # Auto-calculate grid size
        cols = int(np.ceil(np.sqrt(n_images)))
        rows = int(np.ceil(n_images / cols))
    else:
        rows, cols = grid_size
    
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    
    # Handle single image case
    if n_images == 1:
        axes = [axes]
    elif rows == 1 or cols == 1:
        axes = axes.flatten()
    else:
        axes = axes.flatten()
    
    for i in range(n_images):
        # Load image if path is provided
        if isinstance(images[i], str):
            img = Image.open(images[i])
        else:
            img = images[i]
        
        axes[i].imshow(img)
        axes[i].axis('off')
        
        # Add title if provided
        if titles and i < len(titles):
            axes[i].set_title(titles[i])
        else:
            axes[i].set_title(f"Image {i+1}")
    
    # Hide unused subplots
    for i in range(n_images, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

def save_image_with_metadata(image: Image.Image, 
                           output_path: str,
                           metadata: Optional[dict] = None) -> None:
    """
    Save image with optional metadata.
    
    Args:
        image: PIL Image object to save
        output_path: Output file path
        metadata: Optional metadata dictionary
    """
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Save the image
    image.save(output_path, optimize=True, quality=95)
    
    # Save metadata if provided
    if metadata:
        metadata_path = output_path.replace(Path(output_path).suffix, '_metadata.json')
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        print(f"Metadata saved to: {metadata_path}")
    
    print(f"Image saved to: {output_path}")

def convert_image_format(input_path: str, 
                        output_path: str,
                        target_format: str = 'PNG',
                        quality: int = 95) -> None:
    """
    Convert image from one format to another.
    
    Args:
        input_path: Path to input image
        output_path: Path for output image
        target_format: Target format ('PNG', 'JPEG', 'WEBP', etc.)
        quality: Quality for lossy formats (1-100)
    """
    with Image.open(input_path) as img:
        # Convert RGBA to RGB if saving as JPEG
        if target_format.upper() == 'JPEG' and img.mode == 'RGBA':
            # Create white background
            background = Image.new('RGB', img.size, (255, 255, 255))
            background.paste(img, mask=img.split()[-1])  # Use alpha as mask
            img = background
        
        # Save with appropriate parameters
        if target_format.upper() in ['JPEG', 'WEBP']:
            img.save(output_path, format=target_format, quality=quality, optimize=True)
        else:
            img.save(output_path, format=target_format, optimize=True)
    
    print(f"Converted {input_path} to {output_path} ({target_format})")

print("✅ Additional utility functions loaded!")
print("📋 Available utility functions:")
print("- check_image_info(image_path): Get detailed image information")
print("- create_image_grid(images, titles): Display multiple images in grid")
print("- save_image_with_metadata(image, path, metadata): Save image with metadata")
print("- convert_image_format(input_path, output_path, format): Convert image formats")

In [ ]:
# Configuration for output folder
OUTPUT_FOLDER = "output"

# Create output folder if it doesn't exist
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

def get_output_path(filename: str, subfolder: str = "") -> str:
    """
    Generate output path in the output folder.
    
    Args:
        filename: Name of the file
        subfolder: Optional subfolder within output folder
        
    Returns:
        Complete path to the output file
    """
    if subfolder:
        full_path = os.path.join(OUTPUT_FOLDER, subfolder)
        os.makedirs(full_path, exist_ok=True)
        return os.path.join(full_path, filename)
    else:
        return os.path.join(OUTPUT_FOLDER, filename)

def crop_white_background_v2(image_path, output_filename=None, white_threshold=245, padding=10, subfolder=""):
    """
    Updated crop function that saves to output folder by default.
    
    Args:
        image_path: Path to input image
        output_filename: Output filename (will be saved in output folder)
        white_threshold: Threshold for considering pixels as white (0-255)
        padding: Padding to add around the cropped image
        subfolder: Optional subfolder within output folder
    
    Returns:
        Tuple of (cropped_image, output_path)
    """
    # Read image
    img = Image.open(image_path)
    
    # Convert to RGB if needed
    if img.mode != 'RGB':
        img = img.convert('RGB')
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Create mask for non-white pixels
    mask = np.any(img_array < white_threshold, axis=2)
    
    # Find bounding box of non-white pixels
    coords = np.where(mask)
    
    if len(coords[0]) == 0:
        print("Warning: No non-white pixels found. Returning original image.")
        cropped_img = img
    else:
        # Get bounding box coordinates
        top = max(0, coords[0].min() - padding)
        bottom = min(img_array.shape[0], coords[0].max() + padding + 1)
        left = max(0, coords[1].min() - padding)
        right = min(img_array.shape[1], coords[1].max() + padding + 1)
        
        # Crop the image
        cropped_img = img.crop((left, top, right, bottom))
    
    # Generate output filename if not provided
    if output_filename is None:
        input_name = Path(image_path).stem
        output_filename = f"{input_name}_cropped.png"
    
    # Get output path
    output_path = get_output_path(output_filename, subfolder)
    
    # Save the cropped image
    cropped_img.save(output_path)
    print(f"Cropped image saved to: {output_path}")
    
    return cropped_img, output_path

def process_image_v2(input_path, output_filename=None, target_size=None, 
                    white_threshold=245, padding=10, maintain_aspect_ratio=True, subfolder=""):
    """
    Updated process function that saves to output folder by default.
    
    Args:
        input_path: Path to input image
        output_filename: Output filename (will be saved in output folder)
        target_size: Target size for resizing (width, height) or single int
        white_threshold: Threshold for white pixel detection
        padding: Padding around cropped area
        maintain_aspect_ratio: Whether to maintain aspect ratio when resizing
        subfolder: Optional subfolder within output folder
    
    Returns:
        Tuple of (processed_image, output_path)
    """
    print(f"Processing image: {input_path}")
    
    # Step 1: Crop white background
    cropped_img, _ = crop_white_background_v2(input_path, None, white_threshold, padding, "")
    
    original_size = Image.open(input_path).size
    cropped_size = cropped_img.size
    
    print(f"Original size: {original_size}")
    print(f"Cropped size: {cropped_size}")
    
    # Step 2: Resize if target size specified
    if target_size:
        resized_img = resize_image(cropped_img, target_size, maintain_aspect_ratio)
        final_size = resized_img.size
        print(f"Final size: {final_size}")
    else:
        resized_img = cropped_img
        print("No resizing applied")
    
    # Generate output filename if not provided
    if output_filename is None:
        input_name = Path(input_path).stem
        size_suffix = f"_{target_size[0]}x{target_size[1]}" if target_size else ""
        output_filename = f"{input_name}_processed{size_suffix}.png"
    
    # Get output path
    output_path = get_output_path(output_filename, subfolder)
    
    # Save the processed image
    resized_img.save(output_path)
    print(f"Processed image saved to: {output_path}")
    
    return resized_img, output_path

def batch_process_images_v2(input_folder, target_size=None, white_threshold=245, 
                           padding=10, maintain_aspect_ratio=True, subfolder="batch"):
    """
    Updated batch processing function that saves to output folder.
    
    Args:
        input_folder: Path to folder containing input images
        target_size: Target size for resizing (width, height) or single int
        white_threshold: Threshold for white pixel detection
        padding: Padding around cropped area
        maintain_aspect_ratio: Whether to maintain aspect ratio when resizing
        subfolder: Subfolder within output folder for batch results
    
    Returns:
        List of processed image paths
    """
    # Supported image extensions
    image_extensions = {'.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.tif'}
    
    # Get all image files in input folder
    image_files = []
    for file in os.listdir(input_folder):
        if Path(file).suffix.lower() in image_extensions:
            image_files.append(file)
    
    print(f"Found {len(image_files)} images to process")
    print(f"Output folder: {get_output_path('', subfolder)}")
    
    processed_paths = []
    
    # Process each image
    for i, filename in enumerate(image_files):
        input_path = os.path.join(input_folder, filename)
        
        try:
            print(f"\\nProcessing {i+1}/{len(image_files)}: {filename}")
            
            # Generate output filename
            stem = Path(filename).stem
            ext = Path(filename).suffix
            output_filename = f"processed_{stem}{ext}"
            
            processed_img, output_path = process_image_v2(
                input_path=input_path,
                output_filename=output_filename,
                target_size=target_size,
                white_threshold=white_threshold,
                padding=padding,
                maintain_aspect_ratio=maintain_aspect_ratio,
                subfolder=subfolder
            )
            
            processed_paths.append(output_path)
            print(f"✓ Successfully processed: {filename}")
            
        except Exception as e:
            print(f"✗ Error processing {filename}: {str(e)}")
    
    print(f"\\nBatch processing complete!")
    print(f"Processed {len(processed_paths)} images successfully")
    print(f"Results saved in: {get_output_path('', subfolder)}")
    
    return processed_paths

print(f"✅ Output folder configuration loaded!")
print(f"📁 Output folder: {os.path.abspath(OUTPUT_FOLDER)}")
print(f"📋 Updated functions (all save to output folder by default):")
print("- crop_white_background_v2(image_path, output_filename, ...)")
print("- process_image_v2(input_path, output_filename, target_size, ...)")
print("- batch_process_images_v2(input_folder, target_size, ...)")
print("- get_output_path(filename, subfolder): Generate output paths")

In [ ]:
# Updated examples using the new functions that save to output folder

# Example 1: Simple cropping with automatic output to output folder
sample_image_path = "static/images_data/676_test.png"

if os.path.exists(sample_image_path):
    print("=== Example 1: Simple Cropping ===")
    
    # Crop and save to output folder
    cropped_img, output_path = crop_white_background_v2(
        image_path=sample_image_path,
        output_filename="example1_cropped.png",  # Will be saved in output/
        white_threshold=245,
        padding=10
    )
    
    # Display results
    plt.figure(figsize=(12, 6))
    
    # Original image
    plt.subplot(1, 2, 1)
    original_img = Image.open(sample_image_path)
    plt.imshow(original_img)
    plt.title(f"Original Image\\nSize: {original_img.size}")
    plt.axis('off')
    
    # Cropped image
    plt.subplot(1, 2, 2)
    plt.imshow(cropped_img)
    plt.title(f"Cropped Image\\nSize: {cropped_img.size}\\nSaved to: {output_path}")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"✅ Cropped image saved to: {output_path}")
    
else:
    print(f"Sample image not found at: {sample_image_path}")
    print("Available files:")
    for file in glob.glob("static/images_data/*"):
        print(f"  - {file}")

In [ ]:
# Example 2: Complete processing with cropping and resizing
sample_image_path = "static/images_data/676_test.png"

if os.path.exists(sample_image_path):
    print("=== Example 2: Complete Processing ===")
    
    # Process image: crop white background and resize
    processed_img, output_path = process_image_v2(
        input_path=sample_image_path,
        output_filename="example2_processed_800x600.png",
        target_size=(800, 600),
        white_threshold=245,
        padding=10,
        maintain_aspect_ratio=True,
        subfolder="processed"  # Will be saved in output/processed/
    )
    
    # Display comparison
    plt.figure(figsize=(15, 5))
    
    # Original image
    plt.subplot(1, 3, 1)
    original_img = Image.open(sample_image_path)
    plt.imshow(original_img)
    plt.title(f"Original Image\\nSize: {original_img.size}")
    plt.axis('off')
    
    # Cropped only
    plt.subplot(1, 3, 2)
    cropped_only, _ = crop_white_background_v2(sample_image_path, None, 245, 10, "")
    plt.imshow(cropped_only)
    plt.title(f"Cropped Only\\nSize: {cropped_only.size}")
    plt.axis('off')
    
    # Final processed
    plt.subplot(1, 3, 3)
    plt.imshow(processed_img)
    plt.title(f"Processed Final\\nSize: {processed_img.size}")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"✅ Processed image saved to: {output_path}")
    
else:
    print(f"Sample image not found at: {sample_image_path}")
    print("Please check available files in static/images_data/")

In [ ]:
# Example 3: Batch processing with automatic output folder organization
input_folder = "static/images_data/142"  # Folder with many BFI images

if os.path.exists(input_folder):
    print("=== Example 3: Batch Processing ===")
    
    # Process first 5 images as a demo (change this to process all)
    all_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    demo_files = all_files[:5]  # Process first 5 for demo
    
    print(f"Demo: Processing first 5 images from {len(all_files)} available images")
    
    # Create a temporary demo folder
    demo_folder = "demo_input"
    os.makedirs(demo_folder, exist_ok=True)
    
    # Copy first 5 files to demo folder
    for file in demo_files:
        src = os.path.join(input_folder, file)
        dst = os.path.join(demo_folder, file)
        shutil.copy2(src, dst)
    
    # Process the demo batch
    processed_paths = batch_process_images_v2(
        input_folder=demo_folder,
        target_size=(512, 512),  # Resize all to 512x512
        white_threshold=245,
        padding=10,
        maintain_aspect_ratio=True,
        subfolder="batch_demo"  # Will be saved in output/batch_demo/
    )
    
    # Display results
    if processed_paths:
        print(f"\\n📊 Batch Processing Results:")
        print(f"Total processed: {len(processed_paths)}")
        print(f"Output folder: {get_output_path('', 'batch_demo')}")
        
        # Show grid of processed images
        if len(processed_paths) > 0:
            # Load processed images
            processed_images = []
            titles = []
            for path in processed_paths[:6]:  # Show max 6 images
                img = Image.open(path)
                processed_images.append(img)
                titles.append(f"{Path(path).name}\\nSize: {img.size}")
            
            create_image_grid(processed_images, titles, figsize=(15, 10))
    
    # Clean up demo folder
    shutil.rmtree(demo_folder)
    
else:
    print(f"Input folder not found: {input_folder}")
    print("Available folders:")
    for item in os.listdir("static/images_data"):
        item_path = os.path.join("static/images_data", item)
        if os.path.isdir(item_path):
            print(f"  - {item_path}")

In [ ]:
# Summary: Check output folder structure and contents
def show_output_folder_structure():
    """Display the structure of the output folder."""
    print("📁 OUTPUT FOLDER STRUCTURE")
    print("=" * 50)
    
    if not os.path.exists(OUTPUT_FOLDER):
        print(f"Output folder '{OUTPUT_FOLDER}' does not exist yet.")
        return
    
    def print_tree(directory, prefix="", max_depth=3, current_depth=0):
        if current_depth > max_depth:
            return
        
        try:
            items = sorted(os.listdir(directory))
            for i, item in enumerate(items):
                if item.startswith('.'):
                    continue
                    
                item_path = os.path.join(directory, item)
                is_last = i == len(items) - 1
                
                current_prefix = "└── " if is_last else "├── "
                print(f"{prefix}{current_prefix}{item}")
                
                if os.path.isdir(item_path):
                    extension = "    " if is_last else "│   "
                    print_tree(item_path, prefix + extension, max_depth, current_depth + 1)
                else:
                    # Show file size for images
                    try:
                        size = os.path.getsize(item_path)
                        size_mb = size / (1024 * 1024)
                        if size_mb > 1:
                            print(f"{prefix}{'    ' if is_last else '│   '}    ({size_mb:.1f} MB)")
                        else:
                            size_kb = size / 1024
                            print(f"{prefix}{'    ' if is_last else '│   '}    ({size_kb:.1f} KB)")
                    except:
                        pass
        except PermissionError:
            print(f"{prefix}[Permission Denied]")
    
    print(f"{OUTPUT_FOLDER}/")
    print_tree(OUTPUT_FOLDER)
    
    # Count total files and folders
    total_files = 0
    total_folders = 0
    total_size = 0
    
    for root, dirs, files in os.walk(OUTPUT_FOLDER):
        total_folders += len(dirs)
        total_files += len(files)
        for file in files:
            try:
                total_size += os.path.getsize(os.path.join(root, file))
            except:
                pass
    
    print("\\n📊 SUMMARY")
    print("=" * 50)
    print(f"Total folders: {total_folders}")
    print(f"Total files: {total_files}")
    print(f"Total size: {total_size / (1024 * 1024):.1f} MB")
    print(f"Output folder path: {os.path.abspath(OUTPUT_FOLDER)}")

# Function to clean output folder if needed
def clean_output_folder():
    """Clean all contents of the output folder."""
    if os.path.exists(OUTPUT_FOLDER):
        for item in os.listdir(OUTPUT_FOLDER):
            item_path = os.path.join(OUTPUT_FOLDER, item)
            if os.path.isdir(item_path):
                shutil.rmtree(item_path)
            else:
                os.remove(item_path)
        print(f"✅ Cleaned output folder: {OUTPUT_FOLDER}")
    else:
        print(f"Output folder '{OUTPUT_FOLDER}' does not exist.")

# Show current output folder structure
show_output_folder_structure()

print("\\n🛠️  UTILITY FUNCTIONS")
print("=" * 50)
print("- show_output_folder_structure(): Display folder structure")
print("- clean_output_folder(): Clean all output files")
print("- get_output_path(filename, subfolder): Get path in output folder")

# Uncomment the line below to clean the output folder
# clean_output_folder()

In [26]:
def get_haematoxylin_and_eosin_metadata(biosample_id, section_number):
    # Fetch the JSON data
    json_data = fetch_brain_viewer_details(biosample_id)
    if not json_data:
        return None

    # Get HEOS section data safely
    heos_data = json_data.get('thumbNail', {}).get(
        "Haematoxylin and Eosin", [])
    if not heos_data:
        return None

    # Try exact match first
    exact_matches = [item for item in heos_data if str(
        item.get('position_index')) == str(section_number)]
    if exact_matches:
        return exact_matches

    # If not found, try nearest by numeric difference
    try:
        section_number_int = int(section_number)
    except ValueError:
        return None

    valid_indices = [item.get('position_index') for item in heos_data if isinstance(
        item.get('position_index'), int)]
    if not valid_indices:
        return None

    nearest_index = min(
        valid_indices, key=lambda x: abs(x - section_number_int))
    nearest_matches = [item for item in heos_data if item.get(
        'position_index') == nearest_index]

    return nearest_matches if nearest_matches else None

In [27]:
get_haematoxylin_and_eosin_metadata("222", "1")

[{'id': 195846,
  'position_index': 2,
  'jp2Path': '/data/storageIIT/humanbrain/analytics/222/HEOS/B_222_FB74-SL_1-ST_HEOS-SE_2_compressed.jp2',
  'pngPathLow': '/data/storageIIT/humanbrain/analytics/222/HEOS/B_222_FB74-SL_1-ST_HEOS-SE_2_thumbnail.jpg',
  'width': 22664,
  'height': 17694,
  'series': 309,
  'rigidrotation': 180,
  'notes': None,
  'trsdata': '{"center": [30521.670289637936, -16950.44869517583], "rotation": 4.208388660220902}',
  'export_status': 0,
  'rgb_val': None}]

In [28]:
def get_haematoxylin_and_eosin_metadata(biosample_id, section_number):
    # Fetch JSON response using predefined fetcher
    json_data = fetch_brain_viewer_details(biosample_id)
    if not json_data:
        return None

    heos_data = json_data.get('thumbNail', {}).get(
        'Haematoxylin and Eosin', [])
    if not heos_data:
        return None

    # Try exact match first
    exact_match = next(
        (item for item in heos_data if str(
            item.get('position_index')) == str(section_number)),
        None
    )

    match_item = exact_match

    # If exact match not found, find the nearest one by numeric distance
    if not match_item:
        try:
            section_number_int = int(section_number)
        except ValueError:
            return None

        valid_items = [item for item in heos_data if isinstance(
            item.get('position_index'), int)]
        if not valid_items:
            return None

        nearest_item = min(
            valid_items,
            key=lambda item: abs(item['position_index'] - section_number_int)
        )
        match_item = nearest_item

    # Extract only required fields
    filtered = {
        'height': match_item['height'],
        'width': match_item['width'],
        'rotation': match_item.get('rigidrotation', 0),
        'jp2_path_fragment': match_item['jp2Path'],
        'position_index': match_item['position_index']
    }

    return filtered

In [29]:
get_haematoxylin_and_eosin_metadata("222", "1")

{'height': 17694,
 'width': 22664,
 'rotation': 180,
 'jp2_path_fragment': '/data/storageIIT/humanbrain/analytics/222/HEOS/B_222_FB74-SL_1-ST_HEOS-SE_2_compressed.jp2',
 'position_index': 2}

git push -u origin master
git push -u github main

for f in *.jpg; do mv -- "$f" "${f%.jpg}.png"; done
